In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
import csv
import emoji

In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Wojciech\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
dataset = []

In [5]:
with open('../data/reddit_merged.csv', newline='',  encoding="utf8") as data_file:
    reader = csv.reader(data_file, delimiter=',', quotechar='\"')
    for row in reader:
        side_a = row[0]
        side_b = row[1]
        # normalize all whitespaces to space
        side_a = ' '.join(side_a.split())
        side_b = ' '.join(side_b.split())
        dataset.append([side_a, side_b])

In [6]:
# remove duplicates
messages = []
for row in dataset:
    if row[0] not in messages:
        messages.append(row[0])
    if row[1] not in messages:
        messages.append(row[1])

In [7]:
from nltk import sent_tokenize
from nltk.tokenize import TweetTokenizer

tokenized_sentences = []

tokenizer = TweetTokenizer()
for message in messages:
    message =  ' '.join([sentence.lower() for sentence in sent_tokenize(message)])
    message = '<start> ' + message + ' <end>'
    sentences = [emoji.demojize(token) for token in tokenizer.tokenize(message)]
    tokenized_sentences.append([sentences])

In [8]:
corpus = []
for sentences in tokenized_sentences:
    corpus += sentences

In [9]:
corpus[2001]

['<start>',
 "couldn't",
 'find',
 'all',
 'the',
 'ones',
 'i',
 'wanted',
 ',',
 'but',
 'u',
 'was',
 'trying',
 'to',
 'get',
 'moana',
 '<end>']

In [10]:
len(corpus)

62914

In [11]:
import sys
sys.path.insert(0, '../tf_glove')
from tf_glove import GloVeModel

glove_model = GloVeModel(embedding_size=100, context_size=10, min_occurrences=1, learning_rate=0.05, batch_size=512)
glove_model.fit_to_corpus(corpus)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [12]:
glove_model.train(num_epochs=500, log_dir="../glove_logs", summary_batch_interval=5000)

print('words:', len(glove_model.words))

Writing TensorBoard summaries to ../glove_logs
words: 27432


In [13]:
glove_model.save_to_file('../data/glove.local.all.txt')

In [19]:
%matplotlib inline
glove_model.generate_tsne(path="../map_full.png", word_count=len(glove_model.words))